In [19]:
import pickle
import numpy as np
import tensorflow as tf


In [20]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [21]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks): 
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1)) 
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data']
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

train=unpickle('cifar-10-batches-py/data_batch_1')
xtrain=np.array(train[b'data'])
xtrain.shape

(10000, 3072)

In [22]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, 10))
    Yte_onehot = np.zeros((Yte.size, 10))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [23]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
#50000 32 32 3

Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
#10000 32 32 3

# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)
#50000 10
#10000 10

Xtr.shape
Ytr_onehot.shape

(50000, 10)

In [24]:
### placeholder 생성 (X변수, Y변수)

X = tf.placeholder(tf.float32, [None, 32, 32, 3])

Y = tf.placeholder(tf.float32, [None, 10])


drop = tf.placeholder(tf.float32)

In [25]:
### 추가 Layer들

def Layer(node,filtersize,pages,pooling=False,dropout=False,sd=0.01):
    size=int(node.shape[3])
    size2=int(node.shape[2])
    W=tf.Variable(tf.random_normal([3, filtersize, size, pages], stddev=sd))
    L=tf.nn.conv2d(node, W, strides=[1, 1, 1, 1], padding='SAME')
    
    batch_mean,batch_var=tf.nn.moments(L,[0])
    scale=tf.Variable(tf.ones([size2,size2,pages]))
    beta=tf.Variable(tf.zeros([size2,size2,pages]))
    L=tf.nn.batch_normalization(L,batch_mean,batch_var,beta,scale,1e-3)   
    
    L=tf.nn.relu(L)
    if pooling:
        L=tf.nn.max_pool(node, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    if dropout:
        L= tf.nn.dropout(L, drop)
    return L



In [26]:
### Implement the layers of CNNs ###
### 첫 Layer 

first_pages=32
W1 = tf.Variable(tf.random_normal([3,3,3,first_pages], stddev=0.01)) 
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

L01=Layer(L1,3,64,True)

L02=Layer(L01,3,256,False,True)

L_last=L02
print(L_last)


Tensor("dropout_1/mul:0", shape=(?, 8, 8, 256), dtype=float32)


In [27]:
### 3번 Layer: 2차원으로 통합해준다

def FC(node,fc_num):
    size1=int(node.shape[1])
    size2=int(node.shape[2])
    size3=int(node.shape[3])
    nodesize=size1*size2*size3
    for i in range(fc_num):        
        W=tf.Variable(tf.random_normal([nodesize, size3*2], stddev=0.01))#1개의 이미지당 256개 수로 도출시켜주는 weight이다 
        L=tf.reshape(node, [-1, nodesize])#위에서 L2의 사이즈가 8*8*64이므로
        L=tf.matmul(L,W)# ? 8864 X 8864 256
        L=tf.nn.relu(L)
    
    ### 4번 Layer: Fully connected layer
    W2 = tf.Variable(tf.random_normal([size3*2, 10], stddev=0.01))
    # 1의 이미지당 10개 라벨로 도출해주는 weight이다
    model = tf.matmul(L, W2)
    # ? 256 X 256 10 

    return model
    

model=FC(L_last,2)
print(model)


Tensor("MatMul_5:0", shape=(?, 10), dtype=float32)


In [28]:
# cost function, you can change the implementation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

# initialize the variables
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(50000 / batch_size)
total_batch

500

In [ ]:
#gradient,0.001, batch 100

for epoch in range(300):
    total_cost = 0
    
    for i in range(total_batch): #total_batch: 나눠진 등분의 개수
        batch_xs=Xtr[batch_size*(i):batch_size*(i+1)] #batch_size: 한등분의 그림의 개수
        batch_ys=Ytr_onehot[batch_size*(i):batch_size*(i+1)] 
        _, curr_loss = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          drop:0.2})
        total_cost += curr_loss
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    
    if epoch%2==0:
        correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
        accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
        print('Accuracy', sess.run(accuracy, 
                           feed_dict={
                               X:Xte,
                               Y: Yte_onehot,
                           drop:1}))
    

# FC하나 추가, Layer 2로 감소 후 adam,0.001, batch 100
# batchnormalization, dropout0.2


Epoch: 0001 Avg. cost =  0.090
Accuracy 0.7671
Epoch: 0002 Avg. cost =  0.093
Epoch: 0003 Avg. cost =  0.086
Accuracy 0.7708
Epoch: 0004 Avg. cost =  0.094
Epoch: 0005 Avg. cost =  0.090
Accuracy 0.7705
Epoch: 0006 Avg. cost =  0.089
Epoch: 0007 Avg. cost =  0.089
Accuracy 0.7694
Epoch: 0008 Avg. cost =  0.088
